## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Escanaba,US,2021-12-06 17:40:13,45.7453,-87.0646,30.63,75,90,20.71,light snow
1,1,Stonewall,CA,2021-12-06 17:40:13,50.1344,-97.3268,-2.02,81,19,9.57,few clouds
2,2,East London,ZA,2021-12-06 17:39:30,-33.0153,27.9116,68.94,77,20,14.97,few clouds
3,3,Sao Filipe,CV,2021-12-06 17:35:59,14.8961,-24.4956,78.89,62,2,7.94,clear sky
4,4,Rikitea,PF,2021-12-06 17:40:14,-23.1203,-134.9692,75.54,74,93,6.51,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like  for your trip?  "))
max_temp = float(input("What is the maximum temperature you would like for your trip?  "))

What is the minimum temperature you would like  for your trip?  72
What is the maximum temperature you would like for your trip?  92


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities = city_data_df.loc[
                                        (city_data_df["Max Temp"] <= max_temp) &
                                        (city_data_df["Max Temp"] >= min_temp)]
preferred_cities.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Sao Filipe,CV,2021-12-06 17:35:59,14.8961,-24.4956,78.89,62,2,7.94,clear sky
4,4,Rikitea,PF,2021-12-06 17:40:14,-23.1203,-134.9692,75.54,74,93,6.51,overcast clouds
5,5,Isangel,VU,2021-12-06 17:35:25,-19.5500,169.2667,77.25,86,9,5.61,clear sky
7,7,Atuona,PF,2021-12-06 17:33:29,-9.8000,-139.0333,78.89,71,12,22.06,few clouds
8,8,Bintulu,MY,2021-12-06 17:40:15,3.1667,113.0333,75.61,94,40,5.01,scattered clouds
9,9,Comodoro Rivadavia,AR,2021-12-06 17:32:25,-45.8667,-67.5000,78.66,19,0,13.80,clear sky
11,11,Commerce,US,2021-12-06 17:40:15,34.0006,-118.1598,77.99,70,1,0.00,haze
15,15,Nanuque,BR,2021-12-06 17:40:17,-17.8392,-40.3539,82.42,67,85,12.06,light rain
20,20,Georgetown,MY,2021-12-06 17:40:18,5.4112,100.3354,80.89,80,20,3.00,few clouds
21,21,Souillac,MU,2021-12-06 17:40:18,-20.5167,57.5167,79.36,73,40,6.91,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities.isnull().sum()

City_ID                0
City                   0
Country                1
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Sao Filipe,CV,78.89,clear sky,14.8961,-24.4956,
4,Rikitea,PF,75.54,overcast clouds,-23.1203,-134.9692,
5,Isangel,VU,77.25,clear sky,-19.5500,169.2667,
7,Atuona,PF,78.89,few clouds,-9.8000,-139.0333,
8,Bintulu,MY,75.61,scattered clouds,3.1667,113.0333,
9,Comodoro Rivadavia,AR,78.66,clear sky,-45.8667,-67.5000,
11,Commerce,US,77.99,haze,34.0006,-118.1598,
15,Nanuque,BR,82.42,light rain,-17.8392,-40.3539,
20,Georgetown,MY,80.89,few clouds,5.4112,100.3354,
21,Souillac,MU,79.36,scattered clouds,-20.5167,57.5167,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat}, {lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found......\t skipping.")
        

Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.
Hotel not found......	 skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name']!= '']

In [12]:
# 8a. Create the output File (CSV)
output_data_file = 'clean_hotel.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, -80), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))